In [49]:
from bs4 import BeautifulSoup
import requests
import re
import operator
import json
from tabulate import tabulate
import sys
from stop_words import get_stop_words

In [50]:
# get the words

def getwordlist(url):
    word_list = []
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
    for text in soup.find_all('p'):
        if text.text is None:
            continue
        content = text.text
        words = content.lower().split()
        
        for word in words:
            cleaned_word = clean_word(word)
            if len(cleaned_word) > 0:
                word_list.append(cleaned_word)
    
    return word_list
    
# create frequency
def creatFrequencytable(word_list):
    word_count = {}
    for word in word_list:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count

    

In [51]:
# Clean text
def clean_word(word):
    cleaned_word = re.sub('[^A-Za-z]+', '', word)
    return cleaned_word
def remove_stopwords(frequency_list):
    stop_words = get_stop_words('en')
    
    temp_list = []
    for key,value in frequency_list:
        if key not in stop_words:
            temp_list.append([key, value])
    return temp_list
    

In [52]:
string_query = 'DataScience'
search_mode = True

In [53]:
wikipedia_api_link = "https://en.wikipedia.org/w/api.php?format=json&action=query&list=search&srsearch="
wikipedia_link = "https://en.wikipedia.org/wiki/"
url = wikipedia_api_link + string_query

# handle Http request in try block
try:
    response = requests.get(url)
    data = json.loads(response.content.decode('utf-8'))
    wikipedia_page_tag = data['query']['search'][0]['title']
    url = wikipedia_link + wikipedia_page_tag
    page_word_list = getwordlist(url)
    page_word_count = creatFrequencytable(page_word_list)
    sorted_word_frequency_list = sorted(page_word_count.items(), key=operator.itemgetter(1), reverse=True)
    if(search_mode):
        sorted_word_frequency_list = remove_stopwords(sorted_word_frequency_list)
    
    total_words_sum = 0
    for key, value in sorted_word_frequency_list:
        total_words_sum = total_words_sum + value
        
    if len(sorted_word_frequency_list) > 20:
        sorted_word_frequency_list = sorted_word_frequency_list[:20]
        
    final_list = []
    for key, value in sorted_word_frequency_list:
        percentage_value = float(value * 100) / total_words_sum
        final_list.append([key, value, round(percentage_value, 4)])
        
    print_headers = ['Word', 'Frequency', 'Frequency Percentage']
    print(tabulate(final_list, headers=print_headers, tablefmt='orgtbl'))
    
except requests.exceptions.Timeout:
    print("Server Didn't respond, Please try again later")

| Word          |   Frequency |   Frequency Percentage |
|---------------+-------------+------------------------|
| data          |         103 |                 9.5194 |
| science       |          71 |                 6.5619 |
| term          |          15 |                 1.3863 |
| statistics    |          14 |                 1.2939 |
| statistical   |          11 |                 1.0166 |
| big           |           9 |                 0.8318 |
| field         |           8 |                 0.7394 |
| many          |           8 |                 0.7394 |
| now           |           8 |                 0.7394 |
| analytics     |           8 |                 0.7394 |
| scientists    |           8 |                 0.7394 |
| methods       |           7 |                 0.647  |
| analysis      |           7 |                 0.647  |
| business      |           7 |                 0.647  |
| journal       |           7 |                 0.647  |
| computer      |           6 |